In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/GAI Assigment

/content/drive/.shortcut-targets-by-id/1EflCyWvHL3bCw7i4I9zIIKtYM2HaqVhi/GAI Assigment


## 安裝套件
 - transformers (4.37.0) huggingface讀取模型的套件
 - datasets (2.16.1) huggingface讀取資料集的套件
 - torcheval (0.0.7) 各種評價標準

In [ ]:
! pip install transformers
! pip install datasets
! pip install torcheval
! pip install pytorch-ignite

  Using cached torcheval-0.0.7-py3-none-any.whl (179 kB)
  Using cached pytorch_ignite-0.5.0.post2-py3-none-any.whl (296 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using 

In [ ]:
!pip freeze > requirements.txt

# lr = 1e-3

## middle Dim = 50

In [ ]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
# from ignite.metrics import Rouge
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn

class CNNNet(nn.Module):
    def __init__(self, output_dim=1000):
        super(CNNNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 9, 2, 4),
            nn.ReLU(),
            nn.MaxPool2d(4, 4),
            nn.Conv2d(64, 128, 5, 3, 0),
            nn.ReLU(),
            nn.MaxPool2d(8, 8),
        )
        self.linear = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(len(x), -1)

        x = self.linear(x)
        return torch.clip(self.relu(x), 0, 32099)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

class ImageToT5(nn.Module):
    def __init__(self, t5_model_name='t5-small', t5_d_model=50):
        super(ImageToT5, self).__init__()
        self.resnet = CNNNet(output_dim=t5_d_model)
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name, cache_dir="./cache/")

    def generate(self, images, max_length, return_features=False):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5.generate(input_ids=image_features.long(), max_length=max_length)

        if return_features:
            return image_features, outputs
        else:
            return outputs

    def forward(self, images, labels=None):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5(input_ids=image_features.long(), labels=labels)
        return outputs

# 初始化模型
image_to_t5_model = ImageToT5(t5_model_name='google/flan-t5-base', t5_d_model=100).to(device)
tokenizer = T.T5Tokenizer.from_pretrained("google/flan-t5-base", cache_dir="./cache/")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import cv2 as cv
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


In [ ]:
class CommonGenDataset(Dataset):
    def __init__(self, split="train")-> None:
        super().__init__()

        def read_data_path(root):
            data_path = []
            for i in os.listdir(root):
                file = os.path.join(root, i)
                data_path.append(file)

            return data_path

        def load_img(path, path_map_text):
          img = cv.imread(path)
          # img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

          bound = max(img.shape[:2])
          new_img = np.zeros((bound, bound, 3))

          offset_x = (bound - img.shape[0]) // 2
          offset_y = (bound - img.shape[1]) // 2
          new_img[offset_x : img.shape[0] + offset_x, offset_y: img.shape[1] + offset_y, :] = img


          img = new_img
          img = cv.resize(img, (224, 224)).transpose(2,0,1)

          return img, path_map_text["_".join(path.split('/')[-1].split("_")[:3])]

        assert split in ["train", "test"]
        if split == "train":
            self.data_path = read_data_path("./CXIRG_Data/train_data/images")
            data = pd.read_excel("./CXIRG_Data/train_data/reports.xlsx")
        else:
            self.data_path = read_data_path("./CXIRG_Data/valid_data/images")
            data = pd.read_excel("./CXIRG_Data/valid_data/reports.xlsx")

        self.path_map_text = {}
        for name, text in zip(data["name"], data["text"]):
            self.path_map_text[name] = text

        self.data = []
        for path in self.data_path:
          concepts, targets = load_img(path, self.path_map_text)
          self.data.append({"concepts": concepts, "targets":targets, "path": path})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [ ]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = torch.tensor([each["concepts"] for each in sample])
    model_outputs = tokenizer.batch_encode_plus([each["targets"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    paths = [each["path"] for each in sample]
    return model_inputs.to(device), model_outputs["input_ids"].to(device), paths

In [ ]:
from ignite.metrics import Rouge

lr = 1e-5
epochs = 100
optimizer = AdamW(image_to_t5_model.parameters(), lr = 1e-3)
train_batch_size = 8
validation_batch_size = 8

In [ ]:
common_gen_train = DataLoader(CommonGenDataset(split="train"), collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
common_gen_validation = DataLoader(CommonGenDataset(split="test"), collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
rouge = Rouge(variants=["L", 2], multiref="best")

In [ ]:
def evaluate(model):
    pbar = tqdm(common_gen_validation)
    pbar.set_description(f"Evaluating")

    for inputs, targets, _ in pbar:
        output = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(image_to_t5_model.generate(inputs.float(), max_length=1000))]
        targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(targets)]
        for i in range(len(output)):
            sentences = [s.replace('.', ' .').split() for s in output[i]]
            ground_thruths = [t.replace('.', ' .').split() for t in targets[i]]
            for s in sentences:
                rouge.update(([s], [ground_thruths]))
    return rouge.compute()


In [ ]:
def report(images, print_out=False):
  out = image_to_t5_model.generate(images.float(), max_length=1000)[0]
  out = tokenizer.decode(out, skip_special_tokens=True)
  if print_out:
    print(out)
  return out


In [ ]:
for ep in range(epochs):
    pbar = tqdm(common_gen_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets, _ in pbar:
        optimizer.zero_grad()

        loss = image_to_t5_model(images=inputs.float(), labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    if (ep+1) % 20 == 0:
      print(f"answer: {tokenizer.decode(targets[0])}\ngenerate: {report(inputs[:1])}")
      print(f"Rouge-2 score on epoch {ep}:", evaluate(image_to_t5_model))
torch.save(image_to_t5_model, f'd_50.mod')

Training epoch [1/100]:   0%|          | 0/12 [00:00<?, ?it/s]<ipython-input-22-8b3c2dbd574b>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  model_inputs = torch.tensor([each["concepts"] for each in sample])
Training epoch [20/100]: 100%|██████████| 12/12 [00:10<00:00,  1.19it/s, loss=0.0368]


answer: Chest AP view showed:_x000D_ 1.s/p sternotomy and cardiac valve replacement _x000D_ with drainage tubes._x000D_ Enlarged heart size with tortuous aorta._x000D_ 2.Mild R't pneumothorax._x000D_ R't lower lung infiltrations._x000D_ s/p R't chest tube placement._x000D_ 3.No mediastinum widening._x000D_ 4.s/p endotracheal tube and NG intubation._x000D_ s/p R't jugular Swan-Ganz catheterization._x000D_</s>
generate: _x000D_ Chest plain film view showed:_x000D_ Impression:_x000D_ -Suspicious right upper lung patch. _x000D_ -Increased infiltrations. _x000D_ -Tortuous atherosclerotic dilated aorta. _x000D_ -Cardiomegaly. _x000D_ -DJD of spine. _x000D_ -S/P fixation in the left clavicle. _x000D_ -S/P fixation in the left clavicle. _x000D_ -S/P fixation in the left clavicle. _x000D_ -S/P fixation in the left clavicle. _x000D_ -S/P fixation in the left clavicle. _x000D_ -Normal heart size. _x000D_ -DJD of spine. _x000D_ -S/P fixation in left clavicle. _x000D_ -S/P fixation in left clavicle

Evaluating: 100%|██████████| 2/2 [00:08<00:00,  4.26s/it]


Rouge-2 score on epoch 19: {'Rouge-L-P': 0.40937999831802635, 'Rouge-L-R': 0.3975206685772696, 'Rouge-L-F': 0.3975206685772696, 'Rouge-2-P': 0.045560152213378016, 'Rouge-2-R': 0.03680078893139885, 'Rouge-2-F': 0.03680078893139885}


Training epoch [40/100]: 100%|██████████| 12/12 [00:09<00:00,  1.27it/s, loss=0.0592]


...

Rouge-2 score on epoch 79: {'Rouge-L-P': 0.41492080992576197, 'Rouge-L-R': 0.4183470578770737, 'Rouge-L-F': 0.4183470578770737, 'Rouge-2-P': 0.047821256271357075, 'Rouge-2-R': 0.051975588570079154, 'Rouge-2-F': 0.051975588570079154}


Training epoch [100/100]: 100%|██████████| 12/12 [00:10<00:00,  1.20it/s, loss=0.0304]


answer: Chest AP view showed:_x000D_ 1.Normal heart size with tortuous aorta._x000D_ 2.R't middle lung and L't lower lung _x000D_ reticulonodular infiltrations._x000D_ Mild L't pleural effusion._x000D_ 3.No mediastinum widening._x000D_ 4.s/p endotracheal tube and NG intubation._x000D_ s/p R't jugular CVP._x000D_</s>
generate: Chest plain film shows:_x000D_ Impression:_x000D_ -Increased infiltrations in both lungs._x000D_ -Blunting bilateral CP angles._x000D_ Elevated right hemidiaphragm. _x000D_ -Tortuous atherosclerotic dilated aorta._x000D_ -Cardiomegaly._x000D_ -DJD of spine. _x000D_ Mild old fracture of right ribs.


Evaluating: 100%|██████████| 2/2 [00:05<00:00,  2.96s/it]


Rouge-2 score on epoch 99: {'Rouge-L-P': 0.43975675766595557, 'Rouge-L-R': 0.44673377373984585, 'Rouge-L-F': 0.44673377373984585, 'Rouge-2-P': 0.05086771255560593, 'Rouge-2-R': 0.054384495976869715, 'Rouge-2-F': 0.054384495976869715}


In [ ]:
gen = []
ans = []
pth = []
for images, texts, paths in common_gen_validation:
  for image, text, path in zip(images, texts, paths):
    gen.append(report(image.unsqueeze(0)))
    ans.append(tokenizer.decode(text, skip_special_tokens=True))
    pth.append(path)

In [ ]:
df = pd.DataFrame({'path':pth, 'gen': gen, 'ans': ans})
df.to_excel("out.xlsx")

## middle Dim = 100

In [ ]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
# from ignite.metrics import Rouge
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn

class CNNNet(nn.Module):
    def __init__(self, output_dim=1000):
        super(CNNNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 9, 2, 4),
            nn.ReLU(),
            nn.MaxPool2d(4, 4),
            nn.Conv2d(64, 128, 5, 3, 0),
            nn.ReLU(),
            nn.MaxPool2d(8, 8),
        )
        self.linear = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(len(x), -1)

        x = self.linear(x)
        return torch.clip(self.relu(x), 0, 32099)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

class ImageToT5(nn.Module):
    def __init__(self, t5_model_name='t5-small', t5_d_model=50):
        super(ImageToT5, self).__init__()
        self.resnet = CNNNet(output_dim=t5_d_model)
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name, cache_dir="./cache/")

    def generate(self, images, max_length, return_features=False):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5.generate(input_ids=image_features.long(), max_length=max_length)

        if return_features:
            return image_features, outputs
        else:
            return outputs

    def forward(self, images, labels=None):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5(input_ids=image_features.long(), labels=labels)
        return outputs

# 初始化模型
image_to_t5_model = ImageToT5(t5_model_name='google/flan-t5-base', t5_d_model=100).to(device)
tokenizer = T.T5Tokenizer.from_pretrained("google/flan-t5-base", cache_dir="./cache/")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import cv2 as cv
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


In [ ]:
class CommonGenDataset(Dataset):
    def __init__(self, split="train")-> None:
        super().__init__()

        def read_data_path(root):
            data_path = []
            for i in os.listdir(root):
                file = os.path.join(root, i)
                data_path.append(file)

            return data_path

        def load_img(path, path_map_text):
          img = cv.imread(path)
          # img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

          bound = max(img.shape[:2])
          new_img = np.zeros((bound, bound, 3))

          offset_x = (bound - img.shape[0]) // 2
          offset_y = (bound - img.shape[1]) // 2
          new_img[offset_x : img.shape[0] + offset_x, offset_y: img.shape[1] + offset_y, :] = img


          img = new_img
          img = cv.resize(img, (224, 224)).transpose(2,0,1)

          return img, path_map_text["_".join(path.split('/')[-1].split("_")[:3])]

        assert split in ["train", "test"]
        if split == "train":
            self.data_path = read_data_path("./CXIRG_Data/train_data/images")
            data = pd.read_excel("./CXIRG_Data/train_data/reports.xlsx")
        else:
            self.data_path = read_data_path("./CXIRG_Data/valid_data/images")
            data = pd.read_excel("./CXIRG_Data/valid_data/reports.xlsx")

        self.path_map_text = {}
        for name, text in zip(data["name"], data["text"]):
            self.path_map_text[name] = text

        self.data = []
        for path in self.data_path:
          concepts, targets = load_img(path, self.path_map_text)
          self.data.append({"concepts": concepts, "targets":targets, "path": path})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [ ]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = torch.tensor([each["concepts"] for each in sample])
    model_outputs = tokenizer.batch_encode_plus([each["targets"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    paths = [each["path"] for each in sample]
    return model_inputs.to(device), model_outputs["input_ids"].to(device), paths

In [ ]:
from ignite.metrics import Rouge

lr = 1e-5
epochs = 100
optimizer = AdamW(image_to_t5_model.parameters(), lr = 1e-3)
train_batch_size = 8
validation_batch_size = 8

In [ ]:
common_gen_train = DataLoader(CommonGenDataset(split="train"), collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
common_gen_validation = DataLoader(CommonGenDataset(split="test"), collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
rouge = Rouge(variants=["L", 2], multiref="best")

In [ ]:
def evaluate(model):
    pbar = tqdm(common_gen_validation)
    pbar.set_description(f"Evaluating")

    for inputs, targets, _ in pbar:
        output = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(image_to_t5_model.generate(inputs.float(), max_length=1000))]
        targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(targets)]
        for i in range(len(output)):
            sentences = [s.replace('.', ' .').split() for s in output[i]]
            ground_thruths = [t.replace('.', ' .').split() for t in targets[i]]
            for s in sentences:
                rouge.update(([s], [ground_thruths]))
    return rouge.compute()


In [ ]:
def report(images, print_out=False):
  out = image_to_t5_model.generate(images.float(), max_length=1000)[0]
  out = tokenizer.decode(out, skip_special_tokens=True)
  if print_out:
    print(out)
  return out


In [ ]:
for ep in range(epochs):
    pbar = tqdm(common_gen_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets, _ in pbar:
        optimizer.zero_grad()

        loss = image_to_t5_model(images=inputs.float(), labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    if (ep+1) % 20 == 0:
      print(f"answer: {tokenizer.decode(targets[0])}\ngenerate: {report(inputs[:1])}")
      print(f"Rouge-2 score on epoch {ep}:", evaluate(image_to_t5_model))
torch.save(image_to_t5_model, f'd_100.mod')

Training epoch [20/100]: 100%|██████████| 12/12 [00:09<00:00,  1.21it/s, loss=0.0927]


answer: Chest PA View:_x000D_ Impression: _x000D_ > Postinflammatory fibrosis in both upper lungs._x000D_ > Atherosclerosis of aorta._x000D_ > R/O osteoporosis._x000D_ > DJD of spine. _x000D_ > S/P abdominal operation with retained clips.</s>
generate: Chest PA view shows:_x000D_ >No obvious lung mass nor consolidation patch._x000D_ >Normal heart size._x000D_ >No pleural effusion.


Evaluating: 100%|██████████| 2/2 [00:03<00:00,  1.73s/it]


Rouge-2 score on epoch 19: {'Rouge-L-P': 0.5228787878787879, 'Rouge-L-R': 0.4841599914562177, 'Rouge-L-F': 0.4841599914562177, 'Rouge-2-P': 0.08014619883040937, 'Rouge-2-R': 0.06875775296827927, 'Rouge-2-F': 0.06875775296827928}


Training epoch [40/100]: 100%|██████████| 12/12 [00:10<00:00,  1.19it/s, loss=0.074]


...

Rouge-2 score on epoch 79: {'Rouge-L-P': 0.5535261275439702, 'Rouge-L-R': 0.5568169044186783, 'Rouge-L-F': 0.5568169044186783, 'Rouge-2-P': 0.11720191336790255, 'Rouge-2-R': 0.11529602105366256, 'Rouge-2-F': 0.11529602105366256}


Training epoch [100/100]: 100%|██████████| 12/12 [00:10<00:00,  1.18it/s, loss=0.000101]


answer: Chest PA View:_x000D_ Impression: _x000D_ > Reticular infiltration in the bilateral lower lungs._x000D_ > Suspect bilateral lower lung patches. _x000D_ > Bilateral lung emphysematous change._x000D_ > Right apical pleural thickening and fibrosis._x000D_ > Blunting bilateral CP angles. _x000D_ > Borderline cardiomegaly._x000D_ > Atherosclerosis of aorta._x000D_ > R/O osteoporosis._x000D_ > Spondylosis of thoracolumbar spine.</s>
generate: Chest PA view shows:_x000D_ Impression:_x000D_ >Increased bilateral lung markings._x000D_ >Normal heart size. _x000D_ >Atherosclerotic aorta. _x000D_ >Spondylosis._x000D_ >Stent in left axillary region._x000D_ >Calcified nodules in bilateral abdomen, consider bilateral renal stones._x000D_ >Radiopaque lesions in right upper abdomen._x000D_ >S/P NG and endotracheal tube._x000D_ S/P Rt jugular CVC insertion. _x000D_ S/P Rt jugular CVC insertion.


Evaluating: 100%|██████████| 2/2 [00:05<00:00,  2.52s/it]


Rouge-2 score on epoch 99: {'Rouge-L-P': 0.5554061686390931, 'Rouge-L-R': 0.5605571581053206, 'Rouge-L-F': 0.5605571581053206, 'Rouge-2-P': 0.12405145270988412, 'Rouge-2-R': 0.12539961783525821, 'Rouge-2-F': 0.12539961783525821}


In [ ]:
gen = []
ans = []
pth = []
for images, texts, paths in common_gen_validation:
  for image, text, path in zip(images, texts, paths):
    gen.append(report(image.unsqueeze(0)))
    ans.append(tokenizer.decode(text, skip_special_tokens=True))
    pth.append(path)

In [ ]:
df = pd.DataFrame({'path':pth, 'gen': gen, 'ans': ans})
df.to_excel("out1.xlsx")

# lr = 1e-4

## middle Dim = 100

In [ ]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
# from ignite.metrics import Rouge
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn

class CNNNet(nn.Module):
    def __init__(self, output_dim=1000):
        super(CNNNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 9, 2, 4),
            nn.ReLU(),
            nn.MaxPool2d(4, 4),
            nn.Conv2d(64, 128, 5, 3, 0),
            nn.ReLU(),
            nn.MaxPool2d(8, 8),
        )
        self.linear = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(len(x), -1)

        x = self.linear(x)
        return torch.clip(self.relu(x), 0, 32099)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

class ImageToT5(nn.Module):
    def __init__(self, t5_model_name='t5-small', t5_d_model=50):
        super(ImageToT5, self).__init__()
        self.resnet = CNNNet(output_dim=t5_d_model)
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name, cache_dir="./cache/")

    def generate(self, images, max_length, return_features=False):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5.generate(input_ids=image_features.long(), max_length=max_length)

        if return_features:
            return image_features, outputs
        else:
            return outputs

    def forward(self, images, labels=None):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5(input_ids=image_features.long(), labels=labels)
        return outputs

In [ ]:
import cv2 as cv
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


In [ ]:
class CommonGenDataset(Dataset):
    def __init__(self, split="train")-> None:
        super().__init__()

        def read_data_path(root):
            data_path = []
            for i in os.listdir(root):
                file = os.path.join(root, i)
                data_path.append(file)

            return data_path

        def load_img(path, path_map_text):
          img = cv.imread(path)
          # img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

          bound = max(img.shape[:2])
          new_img = np.zeros((bound, bound, 3))

          offset_x = (bound - img.shape[0]) // 2
          offset_y = (bound - img.shape[1]) // 2
          new_img[offset_x : img.shape[0] + offset_x, offset_y: img.shape[1] + offset_y, :] = img


          img = new_img
          img = cv.resize(img, (224, 224)).transpose(2,0,1)

          return img, path_map_text["_".join(path.split('/')[-1].split("_")[:3])]

        assert split in ["train", "test"]
        if split == "train":
            self.data_path = read_data_path("./CXIRG_Data/train_data/images")
            data = pd.read_excel("./CXIRG_Data/train_data/reports.xlsx")
        else:
            self.data_path = read_data_path("./CXIRG_Data/valid_data/images")
            data = pd.read_excel("./CXIRG_Data/valid_data/reports.xlsx")

        self.path_map_text = {}
        for name, text in zip(data["name"], data["text"]):
            self.path_map_text[name] = text

        self.data = []
        for path in self.data_path:
          concepts, targets = load_img(path, self.path_map_text)
          self.data.append({"concepts": concepts, "targets":targets, "path": path})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [ ]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = torch.tensor([each["concepts"] for each in sample])
    model_outputs = tokenizer.batch_encode_plus([each["targets"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    paths = [each["path"] for each in sample]
    return model_inputs.to(device), model_outputs["input_ids"].to(device), paths

In [ ]:
from ignite.metrics import Rouge

lr = 1e-5
epochs = 100
optimizer = AdamW(image_to_t5_model.parameters(), lr = 1e-4)
train_batch_size = 8
validation_batch_size = 8

In [ ]:
common_gen_train = DataLoader(CommonGenDataset(split="train"), collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
common_gen_validation = DataLoader(CommonGenDataset(split="test"), collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
rouge = Rouge(variants=["L", 2], multiref="best")

In [ ]:
def evaluate(model):
    pbar = tqdm(common_gen_validation)
    pbar.set_description(f"Evaluating")

    for inputs, targets, _ in pbar:
        output = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(image_to_t5_model.generate(inputs.float(), max_length=1000))]
        targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(targets)]
        for i in range(len(output)):
            sentences = [s.replace('.', ' .').split() for s in output[i]]
            ground_thruths = [t.replace('.', ' .').split() for t in targets[i]]
            for s in sentences:
                rouge.update(([s], [ground_thruths]))
    return rouge.compute()


In [ ]:
def report(images, print_out=False):
  out = image_to_t5_model.generate(images.float(), max_length=1000)[0]
  out = tokenizer.decode(out, skip_special_tokens=True)
  if print_out:
    print(out)
  return out


In [ ]:
for ep in range(epochs):
    pbar = tqdm(common_gen_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets, _ in pbar:
        optimizer.zero_grad()

        loss = image_to_t5_model(images=inputs.float(), labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    if (ep+1) % 20 == 0:
      print(f"answer: {tokenizer.decode(targets[0])}\ngenerate: {report(inputs[:1])}")
      print(f"Rouge-2 score on epoch {ep}:", evaluate(image_to_t5_model))
torch.save(image_to_t5_model, f'd_100_1e4.mod')

Training epoch [20/100]: 100%|██████████| 12/12 [00:10<00:00,  1.19it/s, loss=0.0317]


answer: Chest AP view showed:_x000D_ 1.Enlarged heart size with tortuous aorta._x000D_ 2.Bilateral lung reticulonodular infiltrations._x000D_ A R't upper lung mass. _x000D_ R't middle and lower lung faint patches._x000D_ 3.No mediastinum widening._x000D_ 4.s/p R't jugular CVP._x000D_</s>
generate: X’en of the on de


Evaluating: 100%|██████████| 2/2 [00:43<00:00, 21.86s/it]


Rouge-2 score on epoch 19: {'Rouge-L-P': 0.4403660863624585, 'Rouge-L-R': 0.43037519860509793, 'Rouge-L-F': 0.43037519860509793, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0}


Training epoch [40/100]: 100%|██████████| 12/12 [00:10<00:00,  1.19it/s, loss=0.00123]


...

Rouge-2 score on epoch 79: {'Rouge-L-P': 0.38676069412776615, 'Rouge-L-R': 0.37523180568766534, 'Rouge-L-F': 0.37523180568766534, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0}


Training epoch [100/100]: 100%|██████████| 12/12 [00:10<00:00,  1.12it/s, loss=0.000122]


answer: Chest AP view shows:_x000D_ Impression:_x000D_ -Bilateral lung centrilobular infiltrations. _x000D_ Pulmonary TB cannot be excluded completely. _x000D_ -Right pleural effusion._x000D_ -The heart shadow is not enlarged._x000D_ -S/P fixation in C-spine._x000D_ -Atherosclerosis of aorta. _x000D_ -Spondylosis.</s>
generate: e


Evaluating: 100%|██████████| 2/2 [00:47<00:00, 23.72s/it]


Rouge-2 score on epoch 99: {'Rouge-L-P': 0.3901181336331149, 'Rouge-L-R': 0.37905744772473365, 'Rouge-L-F': 0.37905744772473365, 'Rouge-2-P': 0.0, 'Rouge-2-R': 0.0, 'Rouge-2-F': 0.0}


In [ ]:
gen = []
ans = []
pth = []
for images, texts, paths in common_gen_validation:
  for image, text, path in zip(images, texts, paths):
    gen.append(report(image.unsqueeze(0)))
    ans.append(tokenizer.decode(text, skip_special_tokens=True))
    pth.append(path)

In [ ]:
df = pd.DataFrame({'path':pth, 'gen': gen, 'ans': ans})
df.to_excel("out1_1e4.xlsx")

# lr = 1e-3 denoised version

## middle Dim = 100

In [ ]:
import transformers as T
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
# from ignite.metrics import Rouge
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import torch
import torch.nn as nn

class CNNNet(nn.Module):
    def __init__(self, output_dim=1000):
        super(CNNNet, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 9, 2, 4),
            nn.ReLU(),
            nn.MaxPool2d(4, 4),
            nn.Conv2d(64, 128, 5, 3, 0),
            nn.ReLU(),
            nn.MaxPool2d(8, 8),
        )
        self.linear = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(len(x), -1)

        x = self.linear(x)
        return torch.clip(self.relu(x), 0, 32099)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

class ImageToT5(nn.Module):
    def __init__(self, t5_model_name='t5-small', t5_d_model=50):
        super(ImageToT5, self).__init__()
        self.resnet = CNNNet(output_dim=t5_d_model)
        self.t5 = T5ForConditionalGeneration.from_pretrained(t5_model_name, cache_dir="./cache/")

    def generate(self, images, max_length, return_features=False):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5.generate(input_ids=image_features.long(), max_length=max_length)

        if return_features:
            return image_features, outputs
        else:
            return outputs

    def forward(self, images, labels=None):
        # 提取图像特征
        image_features = self.resnet(images)

        # 使用T5进行生成任务
        outputs = self.t5(input_ids=image_features.long(), labels=labels)
        return outputs

# 初始化模型
image_to_t5_model = ImageToT5(t5_model_name='google/flan-t5-base', t5_d_model=100).to(device)
tokenizer = T.T5Tokenizer.from_pretrained("google/flan-t5-base", cache_dir="./cache/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import cv2 as cv
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


In [ ]:
class CommonGenDataset(Dataset):
    def __init__(self, split="train")-> None:
        super().__init__()

        def read_data_path(root):
            data_path = []
            for i in os.listdir(root):
                file = os.path.join(root, i)
                data_path.append(file)

            return data_path

        def load_img(path, path_map_text):
          img = cv.imread(path)


          bound = max(img.shape[:2])
          new_img = np.zeros((bound, bound, 3), dtype=img.dtype)

          offset_x = (bound - img.shape[0]) // 2
          offset_y = (bound - img.shape[1]) // 2
          new_img[offset_x : img.shape[0] + offset_x, offset_y: img.shape[1] + offset_y, :] = img


          img = new_img
          img = cv.resize(img, (224, 224))
          img = cv.fastNlMeansDenoisingColored(img,None,10,10,7,21)
          img = img.transpose(2,0,1)

          return img, path_map_text["_".join(path.split('/')[-1].split("_")[:3])]

        assert split in ["train", "test"]
        if split == "train":
            self.data_path = read_data_path("./CXIRG_Data/train_data/images")
            data = pd.read_excel("./CXIRG_Data/train_data/reports.xlsx")
        else:
            self.data_path = read_data_path("./CXIRG_Data/valid_data/images")
            data = pd.read_excel("./CXIRG_Data/valid_data/reports.xlsx")

        self.path_map_text = {}
        for name, text in zip(data["name"], data["text"]):
            self.path_map_text[name] = text

        self.data = []
        for path in self.data_path:
          concepts, targets = load_img(path, self.path_map_text)
          self.data.append({"concepts": concepts, "targets":targets, "path": path})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [ ]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = torch.tensor([each["concepts"] for each in sample])
    model_outputs = tokenizer.batch_encode_plus([each["targets"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    paths = [each["path"] for each in sample]
    return model_inputs.to(device), model_outputs["input_ids"].to(device), paths

In [ ]:
from ignite.metrics import Rouge

lr = 1e-5
epochs = 100
optimizer = AdamW(image_to_t5_model.parameters(), lr = 1e-3)
train_batch_size = 8
validation_batch_size = 8

In [ ]:
common_gen_train = DataLoader(CommonGenDataset(split="train"), collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
common_gen_validation = DataLoader(CommonGenDataset(split="test"), collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)
rouge = Rouge(variants=["L", 2], multiref="best")

In [ ]:
def evaluate(model):
    pbar = tqdm(common_gen_validation)
    pbar.set_description(f"Evaluating")

    for inputs, targets, _ in pbar:
        output = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(image_to_t5_model.generate(inputs.float(), max_length=1000))]
        targets = [re.split(r"[/]", each.replace("<pad>", "")) for each in tokenizer.batch_decode(targets)]
        for i in range(len(output)):
            sentences = [s.replace('.', ' .').split() for s in output[i]]
            ground_thruths = [t.replace('.', ' .').split() for t in targets[i]]
            for s in sentences:
                rouge.update(([s], [ground_thruths]))
    return rouge.compute()


In [ ]:
def report(images, print_out=False):
  out = image_to_t5_model.generate(images.float(), max_length=1000)[0]
  out = tokenizer.decode(out, skip_special_tokens=True)
  if print_out:
    print(out)
  return out


In [ ]:
for ep in range(epochs):
    pbar = tqdm(common_gen_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets, _ in pbar:
        optimizer.zero_grad()

        loss = image_to_t5_model(images=inputs.float(), labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    if (ep+1) % 20 == 0:
      print(f"answer: {tokenizer.decode(targets[0])}\ngenerate: {report(inputs[:1])}")
      print(f"Rouge-2 score on epoch {ep}:", evaluate(image_to_t5_model))
torch.save(image_to_t5_model, f'd_100_denoise.mod')

Training epoch [20/100]: 100%|██████████| 12/12 [00:09<00:00,  1.22it/s, loss=0.0395]


answer: _x000D_ Chest plain film view showed:_x000D_ Impression:_x000D_ -Bilateral apical pleural thickening. _x000D_ -Left lower lung subsegmental atelectasis. _x000D_ -Blunting left CP angle. _x000D_ -Bilateral hilar enlargement. _x000D_ -Normal heart size._x000D_ -DJD of spine. _x000D_ -Mild scoliosis of spine.</s>
generate: Chest plain film view showed:_x000D_ Impression:_x000D_ -Suspicious pulmonary edema. Superimposed pneumonia cannot be excluded. _x000D_ -The mediastinum is not widened._x000D_ -Normal heart size._x000D_ -DJD of spine._x000D_ -S/P NG and endotracheal tube._x000D_ -S/P Rt subclavian CVC insertion.


Evaluating: 100%|██████████| 2/2 [00:24<00:00, 12.00s/it]


Rouge-2 score on epoch 19: {'Rouge-L-P': 0.2696590241034683, 'Rouge-L-R': 0.1657760875177821, 'Rouge-L-F': 0.1657760875177821, 'Rouge-2-P': 0.011959105485675534, 'Rouge-2-R': 0.02395871730092777, 'Rouge-2-F': 0.02395871730092777}


Training epoch [40/100]: 100%|██████████| 12/12 [00:10<00:00,  1.20it/s, loss=0.0019]


...

Rouge-2 score on epoch 79: {'Rouge-L-P': 0.27980921526317964, 'Rouge-L-R': 0.19881659037090774, 'Rouge-L-F': 0.19881659037090774, 'Rouge-2-P': 0.04923144710193638, 'Rouge-2-R': 0.05324556235887756, 'Rouge-2-F': 0.05324556235887756}


Training epoch [100/100]: 100%|██████████| 12/12 [00:09<00:00,  1.20it/s, loss=2.5e-6]


answer: Chest plain film shows:_x000D_ Impression:_x000D_ -Increased infiltrates in both lungs. _x000D_ -Blunting bilateral CP angles. _x000D_ -Cardiomegaly. _x000D_ -Atherosclerosis of aorta. _x000D_ -Scoliosis and DJD of spine. _x000D_ -Bilateral apical pleural thickening. _x000D_</s>
generate: Chest plain film shows:_x000D_ Impression:_x000D_ -Increased infiltrates in both lungs. _x000D_ -Blunting bilateral CP angles. _x000D_ -Cardiomegaly. _x000D_ -Atherosclerosis of aorta. _x000D_ -Scoliosis and DJD of spine. _x000D_ -Bilateral apical pleural thickening. _x000D_


Evaluating: 100%|██████████| 2/2 [00:24<00:00, 12.24s/it]


Rouge-2 score on epoch 99: {'Rouge-L-P': 0.2515772785510445, 'Rouge-L-R': 0.17915200152538638, 'Rouge-L-F': 0.17915200152538638, 'Rouge-2-P': 0.04702450431608154, 'Rouge-2-R': 0.04869058392733173, 'Rouge-2-F': 0.04869058392733173}


In [ ]:
gen = []
ans = []
pth = []
for images, texts, paths in common_gen_validation:
  for image, text, path in zip(images, texts, paths):
    gen.append(report(image.unsqueeze(0)))
    ans.append(tokenizer.decode(text, skip_special_tokens=True))
    pth.append(path)

In [ ]:
df = pd.DataFrame({'path':pth, 'gen': gen, 'ans': ans})
df.to_excel("out1_denoise.xlsx")